In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.ssa.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.util.matrix.*

In [ ]:
// Create model
val model = Network("model")

// Model parameters
val n = 5  // number of items
val m = 2  // cache capacity

// Create nodes
val source = Source(model, "Source")
val cacheNode = Cache(model, "Cache", n, m, ReplacementStrategy.FIFO)
val routerNode = Router(model, "Router")
val delay1 = Delay(model, "Delay1")
val delay2 = Delay(model, "Delay2")
val sink = Sink(model, "Sink")

// Create classes
val jobClass = OpenClass(model, "InitClass", 0)
val hitClass = OpenClass(model, "HitClass", 0)
val missClass = OpenClass(model, "MissClass", 0)

In [18]:
// Set arrival process
source.setArrival(jobClass, Exp(2.0))

// Set service processes
delay1.setService(hitClass, Exp(10.0))
delay1.setService(missClass, Exp(1.0))
delay2.setService(hitClass, Exp(20.0))
delay2.setService(missClass, Exp(2.0))

In [19]:
// Set cache configuration with uniform item references
val probArray = Matrix(1, n).fill(1.0 / n)
val pAccess = DiscreteSampler(probArray)
cacheNode.setRead(jobClass, pAccess)
cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)

In [20]:
// Add links
model.addLink(source, cacheNode)
model.addLink(cacheNode, routerNode)
model.addLink(routerNode, delay1)
model.addLink(routerNode, delay2)
model.addLink(delay1, sink)
model.addLink(delay2, sink)

// Set routing
source.setProbRouting(jobClass, cacheNode, 1.0)
cacheNode.setProbRouting(hitClass, routerNode, 1.0)
cacheNode.setProbRouting(missClass, routerNode, 1.0)
routerNode.setRouting(hitClass, RoutingStrategy.RAND)
routerNode.setRouting(missClass, RoutingStrategy.RAND)
delay1.setProbRouting(hitClass, sink, 1.0)
delay1.setProbRouting(missClass, sink, 1.0)
delay2.setProbRouting(hitClass, sink, 1.0)
delay2.setProbRouting(missClass, sink, 1.0)

In [ ]:
// Solve with CTMC
println("SOLVER: CTMC")
val ctmcSolver = CTMC(model, "keep", false, "cutoff", 1, "seed", 1)
val avgTableCTMC = ctmcSolver.avgNodeTable
avgTableCTMC.print()

In [ ]:
// Reset model and solve with SSA
model.reset()
println("\nSOLVER: SSA")
val ssaSolver = SSA(model, "samples", 10000, "verbose", true, "method", "serial", "seed", 1)
val avgTableSSA = ssaSolver.avgNodeTable
avgTableSSA.print()

In [ ]:
// Reset model and solve with MVA
model.reset()
println("\nSOLVER: MVA")
val mvaSolver = MVA(model, "seed", 1)
val avgTableMVA = mvaSolver.avgNodeTable
avgTableMVA.print()

In [ ]:
// Reset model and solve with NC
model.reset()
println("\nSOLVER: NC")
val ncSolver = NC(model, "seed", 1)
val avgTableNC = ncSolver.avgNodeTable
avgTableNC.print()

// Display cache performance metrics
println("\nHit Ratio: ${cacheNode.hitRatio}")
println("Miss Ratio: ${cacheNode.missRatio}")